______
## Species Ingest

In [9]:
import sqlite3

# Database standard connection
conn=sqlite3.connect('nursery.db')
cursor=conn.cursor()

# Premilinar species data
species_data =     [
    ('Tabebuia rosea', 'Roble Rosado', 'Bignoniaceae', '2025-05-18', 'De 8 a 10 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Tabebuia impetiginosa', 'Roble Morado', 'Bignoniaceae', '2025-05-02', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Laburnum anagyroides', 'Lluvia de Oro', 'Fabaceae', '2025-06-22', 'De 8 a 10 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla-esqueje'),
    ('Mangifera indica', 'Mango', 'Anacardiaceae', '2025-07-06', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Injerto-semilla'),
    ('Delonix regia', 'Acacia', 'Fabaceae', '2025-05-18', 'De 8 a 10 días', 'Rápido', 'Medio', 'Sombra parcial', 'Semilla'),
    ('Citrus × paradisi', 'Toronja', 'Rutaceae', '2025-05-11', 'Más de 15 días', 'Lento', 'Medio', 'Pleno sol', 'Injerto-semilla'),
    ('Carica papaya', 'Papaya', 'Caricaceae', '2025-05-11', 'Menos 8 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dypsis lutescens', 'Palma Areca', 'Arecaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Sombra parcial', 'División-semilla'),
    ('Vachellia farnesiana', 'Aromo', 'Fabaceae', '2025-08-29', 'De 10 a 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dracaena trifasciata', 'Lengua de Suegra', 'Asparagaceae', '2025-08-11', 'Más de 15 días', 'Lento', 'Bajo', 'Pleno sol', 'División-esqueje'),
    ('Terminalia catappa', 'Almendro', 'Combretaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Zamioculcas zamiifolia', 'Millonaria', 'Araceae', '2025-08-29', 'De 10 a 15 días', 'Lento', 'Bajo', 'Sombra parcial', 'División-esqueje'),
    ('Luffa aegyptiaca', 'Estropajo', 'Cucurbitaceae', '2025-08-11', 'De 8 a 10 días', 'Rápido', 'Alto', 'Pleno sol', 'Semilla'),
    ('Cassia grandis', 'Cañandonga', 'Fabaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dracaena marginata', 'Marginata Tricolor', 'Asparagaceae', '2025-09-10', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Esqueje'),
    ('Tradescantia pallida', 'Purpurina', 'Commelinaceae', '2025-09-10', 'Menos 8 días', 'Moderado', 'Bajo', 'Pleno sol', 'Esqueje'),
    ('Tradescantia spathacea', 'Maguey morado', 'Commelinaceae', '2025-09-19', 'De 8 a 10 días', 'Lento', 'Bajo', 'Sombra parcial','División-esquje' ),
    ('Plumeria', 'Bouquet de novia', 'Apocynaceae', '2025-09-19', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol','Semilla-esqueje')
]

# parametrized query for multiple row insertion
insert_query = '''
    INSERT INTO ESPECIE_LOTE
        (NOMBRE_CIENTIFICO, NOMBRE_COMUN, FAMILIA, FECHA_INICIO, GERMINACION_PROMEDIO, CRECIMIENTO, RIEGO, LUZ, PROPAGACION)
    VALUES
        (?,?,?,?,?,?,?,?,?)
'''

# Excecute in transaction
with sqlite3.connect('nursery.db') as conn:
    conn.executemany(insert_query, species_data)

In [ ]:
import sqlite3

def insert_specie(datos_especie):
    """
    Insert a new species to the ESPECIE_LOTE table

    Parameters:
        datos_especie: Tuple with the data to import acording to the table
    """
    try:
        # Connect to the database
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()

        insert_query = ''' 
        INSERT INTO ESPECIE_LOTE 
        (NOMBRE_CIENTIFICO, NOMBRE_COMUN, FAMILIA, FECHA_INICIO, 
        GERMINACION_PROMEDIO, CRECIMIENTO, RIEGO,LUZ, NOTAS)
        VALUES (?,?,?,?,?,?,?,?,?)
        '''
        # Execute the insertion
        cursor.execute(insert_query, datos_especie)
        conn.commit()

        print("Especie insertada correctamente!")

    except sqlite3.Error as error:
        print(f"Error al insertar: {error}")
    
    finally:
        # close connection
        if conn:
            conn.close()

# New species data
new_data = (
    "tradescantia spathacea", # NOMBRE_CIENTIFICO
    "Maguey morado",  # NOMBRE_COMÚN
    "Commelinaceae",     # FAMILIA
    "2025-09-19",      # FECHA_INICIO
    "De 8 a 10 días",  # GERMINACIÓN_PROMEDIO
    "Moderado",          # CRECIMIENTO
    "Bajo",           # RIEGO
    "Pleno sol",       # LUZ
    "Ingreso de 10 plantas" # NOTAS
)
    
insert_specie(new_data)

Especie insertada correctamente!


_______
# Irrigation Ingest

In [ ]:
import sqlite3
from datetime import date

def log_daily_irrigation(irrigation_list, log_date=None):
    """
    A reusable function to log daily irrigation for MULTIPLE lots at once.
    
    Parameters:
    irrigation_list (list of tuples): Each tuple is (lote_id, water_lt, notes).
    log_date (str, optional): Date in 'DD-MM-YYYY' format. Uses today if not provided.
    """
    # Use today's date if no specific date is provided
    if log_date is None:
        log_date = date.today().strftime("%d-%m-%Y")
    
    conn = sqlite3.connect('nursery.db')
    cursor = conn.cursor()
    
    success_count = 0
    total_water = 0.0
    
    try:
        # Calculate total water first
        total_water = sum(water_lt for (lote_id, water_lt, notes) in irrigation_list)
        
        # Prepare the query
        insert_query = '''
            INSERT INTO REGISTRO_RIEGO (ESPECIE_LOTE_ID, FECHA, CANTIDAD_AGUA_LT, NOTAS)
            VALUES (?, ?, ?, ?)
        '''
        
        # Prepare the data: add the same date to every entry
        data_to_insert = [(lote_id, log_date, water_lt, notes) for (lote_id, water_lt, notes) in irrigation_list]
        
        # Execute all inserts at once
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()
        
        success_count = len(irrigation_list)
        print(f"✅ Successfully logged irrigation for {success_count} lots on {log_date}.")
        print(f"💧 Total water used: {total_water} liters")
        
    except sqlite3.Error as e:
        print(f"❌ An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()
    
    return success_count, total_water

# --- HOW TO USE IT DAILY ---
# 1. Define your irrigation for the day in this list
# Format: (lote_id, water_amount, notes)
today_irrigation = [
    (1, 5, None),
    #(2, 5,None),
    (4, 0.5, None),
    #(5, 0.5, None),
    (6, 2, None),
    (7, 0.5, None),
    (8, 1.5, None),
    #(9, 3, None),
    #(10, 2, None),
    (11, 1.5, None),
    (13, 0.5 , None),
    #(14, 0.5, None),
    #(15, 0.5, None),
    #(16, 1, None),
    # Add more lots as needed...
]

# 2. Run this single function call
log_daily_irrigation(today_irrigation)

# --- FOR BACKFILLING PAST DATES ---
# If you need to log for a different date, just provide the date:
# last_weeks_irrigation = [(3, 4.0, ''), (4, 1.5, '')]
# log_daily_irrigation(last_weeks_irrigation, log_date='25-08-2025')

✅ Successfully logged irrigation for 8 lots on 16-09-2025.
💧 Total water used: 17.5 liters


(8, 17.5)

________
#  Batch Observation

In [8]:
import sqlite3
from datetime import date

# Create a function to insert periodical observation data

def log_observation (observations, observation_date=None):
    """ 
    Function for periodical observation for MULTIPLE lots and count follow-up

    Parameters: 
    observation (list of tuples): Each tuple for all observation fields (lote_id, date, plant_qt, height, stage, health, notes)
    date (str): Date in DD-MM-YYYY format. Uses today if not provided.
    """

    # Use today's date if not specific date is provided
    if observation_date is None:
        observation_date = date.today().strftime("%d-%m-%y")

    try:
        # Connect to the database
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()

        # Calculate total plant count
        total_plants = sum(plant_qt for (lote_id, plant_qt, height, stage, health, notes) 
                           in observations)

        # Prepare the query
        insert_query = ''' 
            INSERT INTO OBSERVACION_LOTE (ESPECIE_LOTE_ID, FECHA_OBSERVACION, CANTIDAD_PLANTAS_VIVAS, 
            ALTURA_PROMEDIO_CM, ETAPA_CRECIMIENTO, INDICADOR_SALUD, NOTAS)
            VALUES (?,?,?,?,?,?,?)
        '''

        # Prepare the data: add the same date to every entry
        data_to_insert =  [(lote_id, observation_date, plant_qt, height, stage, health, notes) 
                           for (lote_id, plant_qt, height, stage, health, notes) in observations]

        # Excecute all inserts at once
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()

        print(f"✅Successfully logged {len(observations)} observations")
        print(f"📊 Total living plants")

    except sqlite3.Error as e:
        print(f"❌ An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()
    
    return total_plants
        
    
# --- HOW TO USE IT DAILY ---
if __name__ == "__main__":
    # 1. Define your observations for the day in this list
    # Format: (lote_id, plant_qt, height, stage, health, notes)
    today_observations = [
        
        # Add more lots as needed...
        # (1, 0, 0, '', '', ''),
        # (2, 0, 0, '', '', ''),
        # (3, 0, 0, '', '', ''),
        # (4, 0, 0, '', '', ''),
        # (5, 0, 0, '', '', ''),
        # (6, 0, 0, '', '', ''),
        (6, 76, 20, 'CRECIMIENTO_VEGETATIVO', 'Buena', 'Requerimiento control de plagas y división de 6 papayos'),
        # (7, 0, 0, '', '', ''),
        # (8, 0, 0, '', '', ''),
        # (9, 0, 0, '', '', ''),
        # (10, 0, 0, '', '', ''),
        # (11, 0, 0, '', '', ''),
        # (12, 0, 0, '', '', ''),
        # (13, 0, 0, '', '', ''),
        # (14, 0, 0, '', '', ''),
        # (15, 0, 0, '', '', ''),
        # (16, 0, 0, '', '', ''),
    ]


    # 2. Run this single function call
    result = log_observation(today_observations)
    
    if result is not None:
        print(f"Operation completed successfully. Total plants: {result}")
    else:
        print("Operation failed.")

    # --- FOR BACKFILLING PAST DATES ---
    # If you need to log for a different date, just provide the date:
    # last_week_observations = [
    #     (3, 60, 22, 'Plantula embolsada', 'Regular', 'Necesita más agua'),
    #     (4, 55, 19, 'Plantula embolsada', 'Buena', '')
    # ]
    # log_observation(last_week_observations, observation_date='25-08-2024')

✅Successfully logged 1 observations
📊 Total living plants
Operation completed successfully. Total plants: 76


_____________
# Rain Tracking

As part of the irrigation processes, we consider and document rain as a collaborative force

In [10]:
import sqlite3

# Connect to the existing database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

# Preliminar rain data
rain_data = [
    ('2025-09-21 20:30', '2025-09-21 21:40', 'Llovizna Persistente', '')
]

# Parametrized query for multiple row insertion
insert_query = (''' 
    INSERT INTO REGISTRO_CLIMA 
        (FECHA_HORA_INICIO, FECHA_HORA_FIN, CATEGORIA_PRECIPITACION, NOTAS)
    VALUES
        (?,?,?,?)
''')

# Excecute in transaction
with sqlite3.connect('nursery.db') as conn:
    conn.executemany(insert_query, rain_data)


______
# Pest Control Tracking